### Static/Online methods

In [1]:
import pandas as pd
import numpy as np
import torch

 
from HP_list import Optimal_Stream_algo_HP_dict
from TSB_AD.HP_list import Optimal_Multi_algo_HP_dict
from models.online.feature import Window3D
from utils.stream_analysis import *
from models.online.TimesNet import TimesNet
from models.streaming.MemStream import MemStream
from sklearn import metrics

In [2]:
def run_static_TimesNet(file_name, data):
    Optimal_Det_HP = Optimal_Multi_algo_HP_dict['TimesNet']

    train_index = file_name.split('.')[0].split('_')[-3]
    train_index = int(train_index)
    data_train = data[:train_index, :]
       
    clf = TimesNet(win_size = Optimal_Det_HP['win_size'], lr = Optimal_Det_HP['lr'], enc_in=data_train.shape[1], epochs = 50)
    clf.fit(data_train)
    score = clf.decision_function(data)

    return score
    

def run_online_TimesNet(file_name, data):
    Optimal_Det_HP = Optimal_Multi_algo_HP_dict['TimesNet']
    
    train_index = file_name.split('.')[0].split('_')[-3]
    train_index = int(train_index)
    data_train = data[:train_index, :]
    slidingWindow = Optimal_Det_HP['win_size']

    clf = TimesNet(**Optimal_Det_HP, enc_in=data_train.shape[1], epochs = 50)
    clf.fit(data_train)

    data_stream = Window3D(window = slidingWindow).convert(data) 
    score = np.zeros(len(data))
    for i, data_window in enumerate(data_stream):
            output = clf.decision_function(data_window)
            score[slidingWindow -1 + i]= output[-1]  
    score[: slidingWindow-1] = score[slidingWindow] #padding

    return score

def run_streaming_MemStream(file_name, data):
    Optimal_Det_HP = Optimal_Stream_algo_HP_dict['MemStream']
    
    train_index = file_name.split('.')[0].split('_')[-3]
    train_index = int(train_index)
    data_train = data[:train_index, :]
    slidingWindow = 1

    clf = MemStream(in_dim=data_train.shape[1], **Optimal_Det_HP)
    clf.fit(data_train)

    data_stream = Window3D(window = slidingWindow).convert(data) 
    score = np.zeros(len(data))
    for i, data_window in enumerate(data_stream):
            output = clf.decision_function(data_window)
            score[i]= output

    return score


In [3]:
file_name = '009_MSL_id_8_Sensor_tr_714_1st_1390.csv'
df = pd.read_csv(f"TSB-drift/Datasets/{file_name}").dropna()
label = df['Label'].astype(int).to_numpy()
data = df.iloc[:, 0:-1].values.astype(float)


In [4]:
static_score = run_static_TimesNet(file_name, data)

static_AUC_PR = metrics.average_precision_score(label, static_score)

----- Using GPU NVIDIA RTX 500 Ada Generation Laptop GPU -----


c:\Users\d66285\AppData\Local\miniforge3\envs\venvTSB-AD\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\d66285\AppData\Local\miniforge3\envs\venvTSB-AD\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Valid Epoch [1/50]: 100%|██████████| 1/1 [00:00<00:00, 11.36it/s]


Updating learning rate to 5e-05


Valid Epoch [2/50]: 100%|██████████| 1/1 [00:00<?, ?it/s]


Updating learning rate to 2.5e-05


...

Valid Epoch [9/50]: 100%|██████████| 1/1 [00:00<00:00, 62.50it/s]


EarlyStopping counter: 2 out of 3
Updating learning rate to 1.953125e-07


Valid Epoch [10/50]: 100%|██████████| 1/1 [00:00<?, ?it/s]


EarlyStopping counter: 3 out of 3
   Early stopping<<<


c:\Users\d66285\AppData\Local\miniforge3\envs\venvTSB-AD\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [5]:
online_score = run_online_TimesNet(file_name, data)

online_AUC_PR = metrics.average_precision_score(label, online_score)

----- Using GPU NVIDIA RTX 500 Ada Generation Laptop GPU -----


  0%|          | 0/4 [00:00<?, ?it/s]

Valid Epoch [1/50]: 100%|██████████| 1/1 [00:00<00:00, 536.91it/s]


Updating learning rate to 5e-05


Valid Epoch [2/50]: 100%|██████████| 1/1 [00:00<00:00, 56.75it/s]


...

EarlyStopping counter: 1 out of 3
Updating learning rate to 3.90625e-07


Valid Epoch [9/50]: 100%|██████████| 1/1 [00:00<00:00, 69.40it/s]


EarlyStopping counter: 2 out of 3
Updating learning rate to 1.953125e-07


Valid Epoch [10/50]: 100%|██████████| 1/1 [00:00<00:00, 82.19it/s]
c:\Users\d66285\AppData\Local\miniforge3\envs\venvTSB-AD\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


EarlyStopping counter: 3 out of 3
   Early stopping<<<


In [6]:
streaming_score = run_streaming_MemStream(file_name, data)

streaming_AUC_PR = metrics.average_precision_score(label, streaming_score)

----- Using GPU NVIDIA RTX 500 Ada Generation Laptop GPU -----


Valid Epoch [1/100]: 100%|██████████| 2/2 [00:00<00:00, 333.05it/s, valid_loss=0.319]


Updating learning rate to 0.005


Valid Epoch [2/100]: 100%|██████████| 2/2 [00:00<00:00, 237.00it/s, valid_loss=0.245]


Updating learning rate to 0.0025


...

EarlyStopping counter: 5 out of 7
Updating learning rate to 3.814697265625e-08


Valid Epoch [19/100]: 100%|██████████| 2/2 [00:00<00:00, 876.19it/s, valid_loss=0.169]


EarlyStopping counter: 6 out of 7
Updating learning rate to 1.9073486328125e-08


Valid Epoch [20/100]: 100%|██████████| 2/2 [00:00<00:00, 130.77it/s, valid_loss=0.169]
c:\Users\d66285\Desktop\Code\StrAD\models\streaming\MemStream.py:115: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:212.)
  return torch.as_tensor(x, dtype=torch.float32, device=self.device)


EarlyStopping counter: 7 out of 7
   Early stopping <<<


In [7]:
print("Static AUC-PR performance for TimesNet: ", static_AUC_PR,
      "\nOnline AUC-PR performance for TimesNet: ", online_AUC_PR,
      "\nStreaming AUC-PR performance for MemStream: ", streaming_AUC_PR)

Static AUC-PR performance for TimesNet:  0.0964456603590148 
Online AUC-PR performance for TimesNet:  0.38648989999347055 
Streaming AUC-PR performance for MemStream:  0.018441464295890282
